# Run remote VSCode / code-server on SPCS
- Configure Snowflake Roles & Resources for SPCS
- Configure dockerfile for code-server & non-root miniconda access
- Configure SPCS spec / service file
- Push service file to @specs stage
- Create service from staged spec file



##  Import packages
Note: The following snowflake.core packages are only required if performing DDL operations in python. 

In [2]:

#!/opt/conda/bin/python3
import os

from snowflake.core import Root
from snowflake.core._common import CreateMode
from snowflake.core.warehouse import Warehouse
from snowflake.core.stage import (
    Stage,
    StageEncryption,
    StageDirectoryTable,
)

from snowflake.core.grant import (
    Grant,
    Grantees,
    Privileges,
    Securables,
)

from snowflake.core.role import Role
from snowflake.core.database import Database
from snowflake.connector import connect


###  Optional: Configure connection using local environment vars
* place .env file with env vars in base directory of workspace 
* dotenv loads environment vars into python kernel

In [2]:
#pip install python-dotenv
from dotenv import load_dotenv
import os

ModuleNotFoundError: No module named 'dotenv'

In [1]:

load_dotenv()


NameError: name 'load_dotenv' is not defined

In [5]:
CONNECTION_PARAMETERS_ACCOUNT_ADMIN = {
    "account": os.environ["snowflake_account"],
    "user": os.environ["snowflake_user"],
    "password": os.environ["snowflake_password"]
}


In [9]:

# create a SnowflakeConnection instance
connection_acct_admin = connect(**CONNECTION_PARAMETERS_ACCOUNT_ADMIN)

## Setup

- Note that the following steps can be performed with SQL or Python
- SQL generally more legible / fewer lines of code

In [19]:

try:
    # create a root as the entry point for all object
    root = Root(connection_acct_admin)

    # CREATE ROLE CONTAINER_USER_ROLE
    root.roles.create(Role(
        name='CONTAINER_USER_ROLE',
        comment='My role to use container',
    ))

    # GRANT CREATE DATABASE ON ACCOUNT TO ROLE CONTAINER_USER_ROLE
    # GRANT CREATE WAREHOUSE ON ACCOUNT TO ROLE CONTAINER_USER_ROLE;
    # GRANT CREATE COMPUTE POOL ON ACCOUNT TO ROLE CONTAINER_USER_ROLE;
    # GRANT CREATE INTEGRATION ON ACCOUNT TO ROLE CONTAINER_USER_ROLE;
    # GRANT MONITOR USAGE ON ACCOUNT TO  ROLE  CONTAINER_USER_ROLE;
    # GRANT BIND SERVICE ENDPOINT ON ACCOUNT TO ROLE CONTAINER_USER_ROLE;
    root.grants.grant(Grant(
        grantee=Grantees.role('CONTAINER_USER_ROLE'),
        securable=Securables.current_account,
        privileges=[Privileges.create_database,
                    Privileges.create_warehouse,
                    Privileges.create_compute_pool,
                    Privileges.create_integration,
                    Privileges.monitor_usage,
                    Privileges.bind_service_endpoint
                    ],
    ))

    # GRANT IMPORTED PRIVILEGES ON DATABASE snowflake TO ROLE CONTAINER_USER_ROLE;
    root.grants.grant(Grant(
        grantee=Grantees.role('CONTAINER_USER_ROLE'),
        securable=Securables.database('snowflake'),
        privileges=[Privileges.imported_privileges
                    ],
    ))

    # grant role CONTAINER_USER_ROLE to role ACCOUNTADMIN;
    root.grants.grant(Grant(
        grantee=Grantees.role('ACCOUNTADMIN'),
        securable=Securables.role('CONTAINER_USER_ROLE')
    ))

    # USE ROLE CONTAINER_USER_ROLE
    root.session.use_role("CONTAINER_USER_ROLE")

    # CREATE OR REPLACE DATABASE CONTAINER_HOL_DB;
    root.databases.create(Database(
        name="CONTAINER_HOL_DB",
        comment="This is a Container Quick Start Guide database"
    ), mode=CreateMode.or_replace)

    # CREATE OR REPLACE WAREHOUSE CONTAINER_HOL_WH
    #   WAREHOUSE_SIZE = XSMALL
    #   AUTO_SUSPEND = 120
    #   AUTO_RESUME = TRUE;
    root.warehouses.create(Warehouse(
        name="CONTAINER_HOL_WH",
        warehouse_size="XSMALL",
        auto_suspend=120,
        auto_resume="true",
        comment="This is a Container Quick Start Guide warehouse"
    ), mode=CreateMode.or_replace)

    # CREATE STAGE IF NOT EXISTS specs
    # ENCRYPTION = (TYPE='SNOWFLAKE_SSE');
    root.databases['CONTAINER_HOL_DB'].schemas[CONNECTION_PARAMETERS_ACCOUNT_ADMIN.get("schema")].stages.create(
        Stage(
            name="specs",
            encryption=StageEncryption(type="SNOWFLAKE_SSE")
    ))

    # CREATE STAGE IF NOT EXISTS volumes
    # ENCRYPTION = (TYPE='SNOWFLAKE_SSE')
    # DIRECTORY = (ENABLE = TRUE);
    root.databases['CONTAINER_HOL_DB'].schemas[CONNECTION_PARAMETERS_ACCOUNT_ADMIN.get("schema")].stages.create(
        Stage(
            name="volumes",
            encryption=StageEncryption(type="SNOWFLAKE_SSE"),
            directory_table=StageDirectoryTable(enable=True)
    ))
    # create collection objects as the entry
except:
    pass
finally:
    connection_acct_admin.close()

## Additional Setup Steps (Skipped/pre-existing for the purposes of this demo)
1. setting up security integrations
  - configure ingress -- base case is an oauth config to snowservices_ingress
2. setting up network rule
  - defines ingress/egress IPs, ports
  - typically execute only once, during initial SPCS configuration
3. setting up external access integration
  - param for external access integration is previously created network rule
4. Create image registry within SPCS

Ref: (https://quickstarts.snowflake.com/guide/intro_to_snowpark_container_services/index.html?index=..%2F..index#1)

### Test connection for container user role
- container services are scoped to specific database, schema, and role

In [22]:
CONNECTION_PARAMETERS_CONTAINER_USER_ROLE = {
    "account": os.environ["snowflake_account"],
    "user": os.environ["snowflake_user"],
    "password": os.environ["snowflake_password"],
    "role": "CONTAINER_USER_ROLE",
    "warehouse": "CONTAINER_HOL_WH",
    "Database": "CONTAINER_HOL_DB",
    "Schema": "public"
}

In [23]:
# Connect as CONTAINER_USE_ROLE
connection_container_user_role = connect(**CONNECTION_PARAMETERS_CONTAINER_USER_ROLE)

## Docker Registry Usage
- MFA based login command, below
- Create image registry in prior step, or use existing; see reference (https://docs.snowflake.com/en/developer-guide/snowpark-container-services/working-with-registry-repository)

In [ ]:
! snow spcs image-registry token --format=JSON | \
docker login sfsenorthamerica-demo-cgoyette.registry.snowflakecomputing.com -u 0sessiontoken --password-stdin

## Build docker image - linuxserver.io/code-server
- starting with base image from [lscr.io](https://docs.linuxserver.io/images/docker-code-server/#application-setup)
- added steps for miniconda installation & expose conda on path for non-root users
  - Reference: https://stackoverflow.com/questions/58269375/how-to-install-packages-with-miniconda-in-dockerfile
- see included dockerfile in /src/code-server-2

In [ ]:
# Build the Docker Image -- Run these commands in a terminal
! cd .../sfguide-intro-to-snowpark-container-services/src/code-server-2
## the following command will build an image from the dockerfile in the current directory, and name it cgoyette/code-server-conda
! docker build -t cgoyette/code-server-conda .

In [ ]:
#### Observe Image in docker repo
! docker image list

## Optional / Best practice: Tag image in Docker
- note image ID below, from prior step / local image after pull


In [ ]:
! docker tag cgoyette/code-server-conda sfsenorthamerica-demo-cgoyette.registry.snowflakecomputing.com/container_hol_db/public/image_repo/code-server-conda:amd64-latest

## Docker: push created image to your SPCS registry
After completing this step, use 'docker image list' to check that it exists in your registry

In [ ]:

## note: using tag from prior step
! docker push sfsenorthamerica-demo-cgoyette.registry.snowflakecomputing.com/container_hol_db/public/image_repo/code-server:latest

### Optional: Test running the image, locally
- CG note: Works as expected
- Service available on local host

Note that this requires that the image architecture matches the architecture of your local machine. Docker buildx is also an option, if you have a Mac laptop with an ARM-based M chipset, and want to test locally without managing multiple images with different architectures for testing purposes.

## Configure and Push the Spec YAML
Services in Snowpark Container Services are defined using YAML files. These YAML files configure all of the various parameters, etc. needed to run the containers within your Snowflake account. 

These YAMLs support a large number of configurable parameter.

### Ref/Example: service spec for code server
* see yaml file
* /src/code-server-2/code-server-2.yaml  
* [Service Spec Reference](https://docs.snowflake.com/en/developer-guide/snowpark-container-services/specification-reference)


In [8]:
# push spec file to stage
os.getcwd()

'/Users/cgoyette/Documents/GitHub/sfguide-intro-to-snowpark-container-services'

## Use Snowcli to push the yaml spec to your Database's @specs stage 

Note: if you would like to modify an existing service, take the following steps:
1. Suspend existing/running service
2. Update spec yaml file as needed
3. Perform the following copy operation to overwrite existing spec file
4. Resume service

In [2]:
#! snow object stage copy ./src/code-server-2/code-server-2.yaml @specs --overwrite --connection CONTAINER_hol
! snow object stage copy ./src/code-server-gpu/code-server-gpu.yaml @specs --overwrite --connection CONTAINER_hol



put file:///Users/cgoyette/Documents/GitHub/snowpark-container-services-code-server/src/code-server-gpu/code-server-gpu.yaml @specs auto_compress=false parallel=4 overwrite=True
+------------------------------------------------------------------------------+
|        |        |         |          | source_ | target_c |        |         |
|        |        | source_ | target_s | compres | ompressi |        |         |
| source | target | size    | ize      | sion    | on       | status | message |
|--------+--------+---------+----------+---------+----------+--------+---------|
+------------------------------------------------------------------------------+
|         |         |         |         | source_ | target_ |         |        |
|         |         | source_ | target_ | compres | compres |         | messag |
| source  | target  | size    | size    | sion    | sion    | status  | e      |
|---------+---------+---------+---------+---------+---------+---------+--------|
| code-se | 

## Create & test service

# check status
-- see scratch.sql for sql commands, and to execute from worksheet

In [ ]:
CALL SYSTEM$GET_SERVICE_LOGS('CONTAINER_HOL_DB.PUBLIC.JUPYTER_SNOWPARK_SERVICE', '0', 'jupyter-snowpark',10);

SHOW ENDPOINTS IN SERVICE JUPYTER_SNOWPARK_SERVICE;

    # --- After we make a change to our Jupyter notebook,
    # --- we will suspend and resume the service
    # --- and you can see that the changes we made in our Notebook are still there!
    # ALTER SERVICE CONTAINER_HOL_DB.PUBLIC.JUPYTER_SNOWPARK_SERVICE SUSPEND;

    # ALTER SERVICE CONTAINER_HOL_DB.PUBLIC.JUPYTER_SNOWPARK_SERVICE RESUME;



### Access to running service
The following command will return the exposed endpoint(s) of the service

In [ ]:
#SQL 
SHOW ENDPOINTS IN SERVICE CODE_SERVER_2;


Copy the URL, and paste it in your browser.   
You will be asked to login to your Snowflake, after which you should successfully see your code-server instance running, all inside of Snowflake!   

Note, to access the service the user logging in must have the CONTAINER_USER_ROLE AND their default role cannot be ACCOUNTADMIN, SECURITYADMIN, or ORGADMIN.

### Next steps / Testing: Upload and Modify a Notebook
CG Note: the following is lifted from the quickstart, but the behavior described has been validated, e.g.,
- One does not need to suspend or resume an SPCS service to observe new files copied to stage
- **Note: in service spec yaml, set the user ID and group ID to the default UID/GID for the hosted service. For code-server, this is 911 for both.** This enables r/w access to/from the stage and files accessible via the service.

---

Notice that in our spec YAML file we mounted the @volumes/jupyter-snowpark internal stage location to our workspace/stage directory inside of our running container. 

What this means is that we will use our internal stage @volumes to persist and store artifacts from our container. If you go check out the @volumes stage in Snowsight, you'll see that when we created our jupyter_snowpark_service, a folder was created in our stage: @volumes/jupyter-snowpark

Now, any file that is uploaded to @volumes/jupyter-snowpark will be available inside of our container in the /home/jupyter directory, and vice versa. Read more about volume mounts in the documentation. To test this out, let's upload the sample Jupyter notebook that is in our source code repo at .../sfguide-intro-to-snowpark-container-services/src/jupyter-snowpark/sample_notebook.ipynb. To do this you can either  
1. Click on the jupyter-snowpark directory in Snowsight, click the blue + Files button and drag/browse to sample_notebook.ipynb. Click Upload. Navigate to your Jupyter service UI in your browser, click the refresh arrow and you should now see your notebook available!  OR   
2. Upload sample_notebook.ipynb to @volumes/jupyter-snowpark using SnowCLI OR 
3. Upload sample_notebook.ipynb directly in your Jupyter service on the home screen by clicking the Upload button. If you now navigate back to @volumes/jupyter-snowpark in Snowsight, our run an ls @volumes/jupyter-snowpark SQL command, you should see your sample_notebook.ipynb file listed. Note you may need to hit the Refresh icon in Snowsight for the file to appear.  

What we've done is now created a Jupyter notebook which we can modify in our service, and the changes will be persisted in the file because it is using a stage-backed volume. Let's take a look at the contents of our sample_notebook.ipynb. Open up the notebook in your Jupyter service: